In [1]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Datasets/IMDB_Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
tokenize = Tokenizer()
tokenize.fit_on_texts(df['review'])
X_seq = tokenize.texts_to_sequences(df['review'])
X_pad = pad_sequences(X_seq, maxlen = 500)

In [6]:
vocab_size = len(tokenize.word_index) + 1

In [7]:
tokenize2 = Tokenizer()
tokenize2.fit_on_texts(df['sentiment'])
Y_seq = tokenize2.texts_to_sequences(df['sentiment'])
Y_pad = pad_sequences(Y_seq, maxlen = 1)

In [8]:
Y = df['sentiment'].replace({'positive' : 1, 'negative': 0})

In [10]:
model_lstm = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, 40, input_length = 500),
        keras.layers.Dropout(0.5),
        keras.layers.LSTM(100),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation = 'sigmoid')
])

model_lstm.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 40)           4970120   
_________________________________________________________________
dropout (Dropout)            (None, 500, 40)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 5,026,621
Trainable params: 5,026,621
Non-trainable params: 0
_________________________________________________________________


In [11]:
model_lstm.fit(X_pad, Y, validation_split = 0.2, batch_size = 16, epochs = 10)

Epoch 1/10
2500/2500 [==============================] - 193s 74ms/step - loss: 0.4981 - accuracy: 0.7520 - val_loss: 0.3611 - val_accuracy: 0.8508
Epoch 2/10
2500/2500 [==============================] - 185s 74ms/step - loss: 0.2467 - accuracy: 0.9075 - val_loss: 0.3138 - val_accuracy: 0.8765
Epoch 3/10
2500/2500 [==============================] - 185s 74ms/step - loss: 0.1829 - accuracy: 0.9337 - val_loss: 0.3136 - val_accuracy: 0.8804
Epoch 4/10
2500/2500 [==============================] - 184s 74ms/step - loss: 0.1230 - accuracy: 0.9598 - val_loss: 0.3310 - val_accuracy: 0.8925
Epoch 5/10
2500/2500 [==============================] - 183s 73ms/step - loss: 0.0923 - accuracy: 0.9697 - val_loss: 0.3862 - val_accuracy: 0.8629
Epoch 6/10
2500/2500 [==============================] - 184s 74ms/step - loss: 0.0770 - accuracy: 0.9745 - val_loss: 0.3368 - val_accuracy: 0.8865
Epoch 7/10
2500/2500 [==============================] - 184s 74ms/step - loss: 0.0510 - accuracy: 0.9845 - val_loss: 0

In [14]:
model_lstm.save("movie_sentiment_analysis_lstm.h5")

with open("movie_sentiment_analysis_lstm.json", "w") as json_file:
  model_json = model_lstm.to_json() 
  json_file.write(model_json) 

# Saving weights of the model to a HDF5 file 
model_lstm.save_weights("movie_sentiment_analysis_lstm_weights.h5")

In [15]:
model_gru = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, 40, input_length = 500),
        keras.layers.Dropout(0.5),
        keras.layers.GRU(100),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation = 'sigmoid')
])

model_gru.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_gru.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 40)           4970120   
_________________________________________________________________
dropout_4 (Dropout)          (None, 500, 40)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               42600     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 5,012,821
Trainable params: 5,012,821
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_gru.fit(X_pad, Y, validation_split = 0.2, batch_size = 16, epochs = 5)

Epoch 1/5
2500/2500 [==============================] - 179s 71ms/step - loss: 0.5039 - accuracy: 0.7287 - val_loss: 0.2484 - val_accuracy: 0.8979
Epoch 2/5
2500/2500 [==============================] - 175s 70ms/step - loss: 0.1844 - accuracy: 0.9307 - val_loss: 0.2522 - val_accuracy: 0.9085
Epoch 3/5
2500/2500 [==============================] - 173s 69ms/step - loss: 0.1032 - accuracy: 0.9648 - val_loss: 0.2358 - val_accuracy: 0.9084
Epoch 4/5
2500/2500 [==============================] - 176s 70ms/step - loss: 0.0644 - accuracy: 0.9783 - val_loss: 0.2945 - val_accuracy: 0.9067
Epoch 5/5
2500/2500 [==============================] - 176s 70ms/step - loss: 0.0434 - accuracy: 0.9856 - val_loss: 0.3470 - val_accuracy: 0.9027


In [17]:
model_gru.save("movie_sentiment_analysis_gru.h5")

with open("movie_sentiment_analysis_gru.json", "w") as json_file:
  model_gru_json = model_gru.to_json() 
  json_file.write(model_gru_json) 

# Saving weights of the model to a HDF5 file 
model_gru.save_weights("movie_sentiment_analysis_gru_weights.h5")

In [19]:
model_bilstm = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, 40, input_length = 500),
        keras.layers.Dropout(0.5),
        keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences = True)),
        keras.layers.Bidirectional(keras.layers.LSTM(100)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation = 'sigmoid')
])

model_bilstm.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_bilstm.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 40)           4970120   
_________________________________________________________________
dropout_7 (Dropout)          (None, 500, 40)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 500, 200)          112800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               240800    
_________________________________________________________________
dropout_8 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 5,323,921
Trainable params: 5,323,921
Non-trainable params: 0
____________________________________________

In [20]:
model_bilstm.fit(X_pad, Y, validation_split = 0.2, batch_size = 16, epochs = 5)

Epoch 1/5
2500/2500 [==============================] - 398s 157ms/step - loss: 0.5488 - accuracy: 0.7017 - val_loss: 0.3518 - val_accuracy: 0.8507
Epoch 2/5
2500/2500 [==============================] - 394s 158ms/step - loss: 0.2571 - accuracy: 0.8992 - val_loss: 0.3835 - val_accuracy: 0.8522
Epoch 3/5
2500/2500 [==============================] - 393s 157ms/step - loss: 0.1609 - accuracy: 0.9410 - val_loss: 0.2411 - val_accuracy: 0.9036
Epoch 4/5
2500/2500 [==============================] - 395s 158ms/step - loss: 0.1037 - accuracy: 0.9646 - val_loss: 0.2750 - val_accuracy: 0.8983
Epoch 5/5
2500/2500 [==============================] - 396s 159ms/step - loss: 0.0705 - accuracy: 0.9759 - val_loss: 0.2912 - val_accuracy: 0.9023


In [21]:
model_bilstm.save("movie_sentiment_analysis_bilstm.h5")

with open("movie_sentiment_analysis_bilstm.json", "w") as json_file:
  model_gru_json = model_bilstm.to_json() 
  json_file.write(model_gru_json) 

# Saving weights of the model to a HDF5 file 
model_bilstm.save_weights("movie_sentiment_analysis_bilstm_weights.h5")

In [ ]:
# Without dropout
model.fit(X_pad, Y, validation_split = 0.2, batch_size = 16, epochs = 10)

Epoch 1/10
2500/2500 [==============================] - 176s 70ms/step - loss: 0.5324 - accuracy: 0.7258 - val_loss: 0.4065 - val_accuracy: 0.8321
Epoch 2/10
2500/2500 [==============================] - 173s 69ms/step - loss: 0.3257 - accuracy: 0.8685 - val_loss: 0.2753 - val_accuracy: 0.8861
Epoch 3/10
2500/2500 [==============================] - 173s 69ms/step - loss: 0.1426 - accuracy: 0.9504 - val_loss: 0.2621 - val_accuracy: 0.9002
Epoch 4/10
2500/2500 [==============================] - 172s 69ms/step - loss: 0.0797 - accuracy: 0.9739 - val_loss: 0.3029 - val_accuracy: 0.8999
Epoch 5/10
2500/2500 [==============================] - 172s 69ms/step - loss: 0.0457 - accuracy: 0.9860 - val_loss: 0.3759 - val_accuracy: 0.8929
Epoch 6/10
2500/2500 [==============================] - 172s 69ms/step - loss: 0.0236 - accuracy: 0.9937 - val_loss: 0.5010 - val_accuracy: 0.8904
Epoch 7/10
2500/2500 [==============================] - 171s 68ms/step - loss: 0.0135 - accuracy: 0.9965 - val_loss: 0